# Bibliotecas Necessárias

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier as mlp

# Importando a Base de Dados

In [2]:
data = pd.read_csv('C:\\Users\\LENOVO\\Desktop\\UFPI\\LINA\\PIBIC 2019-2020 (01.08.2019 a 31.07.2020)\\Bases\\Bases Numéricas Discretizadas\\base_limpa_numerica_discretizadaEWD_6atributos.csv')

In [3]:
data.head()

,Matriz_mesangial_aspecto_nodular,Matriz_mesangial_aumentada,Matriz_mesangial_aumento_discreto,Matriz_mesangial_mesangiolise,Matriz_mesangial_normal,Celularidade_mesangial_aumentada,Celularidade_mesangial_diminuida,Celularidade_mesangial_normal,Membrana_basal_glomerular_duplicacao,Membrana_basal_glomerular_duplicacao_e_espiculas,...,Clinica_sindrome_nefrotica,Clinica_sindrome_nefrotica_cortico-dependente,Clinica_sindrome_nefrotica_cortico-resistente,Diagnóstico,Media,Desvio,Descritor de Suavidade Relativa,Terceiro Momento,Uniformidade,Entropia
0,0,0,0,0,1,0,0,1,0,0,...,1,0,0,Doença de Lesões mínimas,1.0,0.0,1.0,0.0,0.0,2.0
1,0,0,0,0,1,0,0,1,0,0,...,0,0,0,Rim normal,2.0,1.0,0.0,0.0,0.0,2.0
2,0,1,0,0,0,0,0,1,0,0,...,1,0,0,Glomeruloesclerose segmentar e focal,2.0,1.0,0.0,1.0,0.0,2.0
3,0,0,0,0,1,0,0,1,0,0,...,1,0,0,Glomerulopatia membranosa primária,2.0,1.0,0.0,0.0,0.0,2.0
4,0,0,1,0,0,0,0,1,1,0,...,1,0,0,Glomerulopatia membranosa primária,0.0,0.0,0.0,0.0,2.0,1.0


# Aplicando o Algoritmo de Clusterização

In [4]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++', n_init = 10, random_state = 1234)

In [5]:
kmeans.fit(data.drop(['Diagnóstico'], axis = 1))

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=1234, tol=0.0001, verbose=0)

In [6]:
kmeans.labels_

array([0, 0, 2, 0, 3, 3, 3, 0, 3, 3, 3, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2,
       2, 2, 0, 2, 2, 2, 1, 0, 0, 3, 3, 3, 0, 2, 2, 2, 4, 4, 4, 4, 0, 4,
       0, 0, 0, 0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 4, 1, 0, 4, 4, 4, 4,
       0, 0, 4, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# Agrupando a Base de Acordo com os Grupos Criados

In [7]:
def grouping(data, labels, atributo_classe):
    # criando a nova coluna para a base:
    data['grupo'] = data[atributo_classe]
    
    # preenchendo a nova coluna com o respectivo identificador do grupo:
    for i in range(data.shape[0]):
        data['grupo'][i] = labels[i]
    
    frames = []
    # agrupando a base de acordo com os grupos formados:
    for clt, grupo in data.drop([atributo_classe], axis = 1).groupby('grupo') :
        frames.append(grupo)
    
    return frames

In [8]:
data_agrupada = grouping(data, kmeans.labels_, 'Diagnóstico')
# Cada um dos grupos é um DataFrame com um valor único para o atributo grupo. 

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


# Aplicando Algoritmo de Classificação

In [9]:
def trainTest(data, coluna, porc_teste):
    # Vale lembrar que X representa todas as colunas que serão utilizadas para predizer valores da coluna Y.
    X = data.drop(data.columns[coluna], axis=1).values
    Y = data.loc[:,data.columns[coluna]].values
    
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = porc_teste)
    
    return x_train, x_test, y_train, y_test

In [10]:
def classificar(metodo, data, porc_treino, dobras):
    acuracia = [0]*data.shape[1]
    # acuracia é uma lista cuja quantidade de elementos é igual à quantidade de colunas do dataframe (data.shape[1])
    # e que possui, até o momento, todos os seus elementos sendo 0 (o número inteiro 0).
    porc_teste = ((100 - porc_treino)/100)
    
    for i in range(dobras):
        # para cada uma das camadas da MLPC
        for coluna in range(data.shape[1]):
            x_train, x_test, y_train, y_test = trainTest(data, coluna, porc_teste)
            # temos a base particionada em treino e teste levando em consideração como atributo classe a coluna em quetão
            
            y_train = np.asarray(y_train, dtype="|S6")
            # convertemos y_train a uma matriz, especificando que o tipo de dado encontrado em y_train é string
            y_test = np.asarray(y_test, dtype="|S6")
            # convertemos y_test a uma matriz, especificando que o tipo de dado encontrado em y_test é string
            
            if x_train.size == 0:
                acuracia[coluna] += 0
                
            else:
                metodo.fit(x_train, y_train)
                acuracia[coluna] += metodo.score(x_test, y_test)
                
    acuracia = [i/dobras for i in acuracia]
    # após o fim do cálculo da acurácia para cada uma das colunas do dataframe já sem o atributo classe, temos em
    # cada uma das posições da lista acuracia a soma do score obtido para essa posição em cada uma das camadas da
    # rede neural. Para termos um valor médio, dividimos o valor de cada uma dessas posições pelo número total de
    # camadas da rede neural.
    
    resultado = list(zip(data.columns, acuracia))
    # A função zip () pega iteráveis (pode ser zero ou mais), agrega-os em uma tupla e a retorna. Colocando-o dentro
    # do método list, retornamos uma lista onde cada elemento é uma tupla. O primeiro elemento da tupla é o nome da
    # coluna e o segundo elemento da tupla é a sua respectiva acurácia.
    
    return resultado

In [11]:
def classification(data_agrupada, atributo_classe, porc_treino, dobras):
    result = []
    for grupo in data_agrupada:
        # grupo é um DataFrame com todos os registros do nosso DataFrame original que têm o mesmo valor na coluna grupo
        cluster = grupo[atributo_classe].unique()
        # valor assumido pelo atributo classe do grupo atual
        
        classificacao = classificar(mlp(max_iter=2000, random_state = 1234), grupo.drop([atributo_classe], axis = 1), porc_treino, dobras)
        result.append((cluster, classificacao))
    
    return result

In [12]:
classificacao = classification(data_agrupada, 'grupo', 60, 10)
# classificacao é uma lista de tuplas. O primeiro elemento de cada uma das tuplas é o valor assumido pelo atributo classe 
# em um determinado grupo. O segundo elemento é também uma lista de tuplas, onde o primeiro elemento da tupla é uma das 
# colunas do db (com execeção do atributo classe, que já foi retirado) e o segundo elemento é a acurácia dessa coluna 
# (classificacao). Em outras palavras temos uma lista que informa um valor assumido pelo atributo classe e as 
# correspondentes acurácias assumidas pelas outras colunas da dataframe quanto a esse valor do atributo classe.

In [13]:
# Veja: 
classificacao

[(array([0], dtype=object),
  [('Matriz_mesangial_aspecto_nodular', 1.0),
   ('Matriz_mesangial_aumentada', 0.86),
   ('Matriz_mesangial_aumento_discreto', 1.0),
   ('Matriz_mesangial_mesangiolise', 1.0),
   ('Matriz_mesangial_normal', 0.8300000000000001),
   ('Celularidade_mesangial_aumentada', 0.79),
   ('Celularidade_mesangial_diminuida', 1.0),
   ('Celularidade_mesangial_normal', 0.85),
   ('Membrana_basal_glomerular_duplicacao', 0.9200000000000002),
   ('Membrana_basal_glomerular_duplicacao_e_espiculas', 1.0),
   ('Membrana_basal_glomerular_duplicacao_espiculas_e_ruptura', 1.0),
   ('Membrana_basal_glomerular_espessada', 0.9400000000000001),
   ('Membrana_basal_glomerular_espessamento', 1.0),
   ('Membrana_basal_glomerular_espessamento_duplicacao_espiculas', 0.93),
   ('Membrana_basal_glomerular_espiculas', 1.0),
   ('Membrana_basal_glomerular_normal', 0.8700000000000001),
   ('Membrana_basal_glomerular_ruptura', 0.95),
   ('Membrana_basal_glomerular_ruptura_buracos_e_duplicacao',

# Rotulando os Grupos Criados

In [14]:
def rotulador(grupo, class_info, V):
    medias = [(i, acuracia*100) for i, acuracia in class_info]
    # é uma lista de tuplas em que o primeiro elemento de cada tupla é uma das colunas do dataframe e o segundo ele-
    # mento é a sua respectiva acurácia. Ao mutiplicarmos a acurácia por 100, temos o seu valor em procentagem.
    
    medias.sort(key = lambda x: x[1], reverse = True)
    # método que ordena uma lista de forma crescente (reverse = False) ou decrescente (reverse = True). O parâmetro
    # key serve para designar uma função que será o critério de ordenação. No caso, estamos especificando através
    # da função lamba que o o segundo elemento da lista de tuplas é o que deve ser analizado para a ordenação. Esse
    # segundo elemento trata-se justamente da acurácia daquela coluna para o valor assumido pelo atributo classe
    # no grupo específico.
    
    minimo = medias[0][1] - V
    # valor mínimo aceitável para que o valor de um atributo qualquer seja considerado como algo que caracteriza os
    # elementos daquele grupo (ou seja, para que esse valor seja considerado como um rótulo). No caso, especificamos
    # o valor mínimo subtraindo do valor da maior acurácia o valor de V (no caso, é 10).
    
    titulos = grupo.columns.values.tolist()
    # convertemos uma Serie em uma Lista. No caso, nossa lista será formada pelo nome das colunas/atributos do nosso
    # dataframe (do nosso grupo em questão), com exceção do atributo classe, pois já o retiramos na chamada dessa função.
    
    rotulos = []
    
    for i in range(grupo.shape[1]):
        # para cada um das colunas do nosso drataframe/grupo
        if medias[i][1] >= minimo:
            # se a acurácia for maior ou igual ao limite mínimo estipulado
            coluna = medias[i][0]
            # nome da coluna cuja a acurácia é minimamente aceitável.
            
            valor_mais_comum = grupo[coluna].mode()[0]
            
            rotulo = (coluna, valor_mais_comum, round(medias[i][1], 2))
            
            rotulos.append(rotulo)
            
    return rotulos

In [15]:
def rotular(data_agrupada, atributo_classe, classificacao):
    rotulo = []
    for grupo in data_agrupada:
        # grupo é um DataFrame com todos os registros do nosso DataFrame original que têm o mesmo valor na coluna grupo
        cluster = grupo[atributo_classe].unique()
        # valor assumido pelo atributo classe do grupo atual
        
        classe_info = [i[1] for i in classificacao if i[0]== cluster][0]
        # o for serve para percorremos todos os elementos da lista de classificação em busca daquele cujo o primeiro
        # elemento da tupla é igual ao valor assumido pelo atributo classe do grupo em questão, pois é em relação a
        # esse grupo que queremos fazer a rotulação. i[0] representa o primeiro elemento da lista de tuplas classifi-
        # cacao_infor, que é mum dos possíveis valores a serem assumidos por um grupo. Caso encontremos o grupo que 
        # estamos procurando, pegamos o segundo elemento da tupla (i[1]) que também é uma lista de tuplas e, após isso, 
        # selecionamos o primeiro elemento dessa lista ( [i[1]][0] ) que é uma lista de tuplas que têm como primeiro 
        # elemento uma das outras colunas do dataframe e o segundo elemento a sua respectiva acurácia. 
        # Para  nos utilizarmos do recurso i[1] for i in classificacao_infor if i[0]==clt (chamaremos de x) sem dar erro 
        # de sintaxe, precisamos usar os colchetes envolvendo x. Isso acaba criando uma lista. Como o retorno da operação 
        # x, agora entre colchetes, já retorna uma lista, utilizamos o [0] ao lado de x para retirarmos os colchetes que 
        # colocamos anteriormente.
        rotulo.append((cluster, rotulador(grupo.drop([atributo_classe], axis = 1), classe_info, 0)))
    
    return rotulo

In [16]:
rotulos = rotular(data_agrupada, 'grupo', classificacao)

# rotulos é uma lista de tuplas. Cada uma das tuplas tem como primeiro elemento um array contendo um único valor que é o
# valor assumido pelo atributo classe naquele grupo. O segundo elemento da tupla também é uma lista de tuplas. Nessa lista
# temos todos ou outros atributos do DataFrame que possuem acurácia maior ou igual ao mínimo estipulado para ser caracteri
# zado como um rótulo do grupo. O primeiro elemento de cada uma das tuplas dessa lista é justamente o nome de um desses 
# atributos. O segundo é o valor mais comum que aparece naquele atributo.

In [17]:
rotulos

[(array([0], dtype=object),
  [('Matriz_mesangial_aspecto_nodular', 0, 100.0),
   ('Matriz_mesangial_aumento_discreto', 0, 100.0),
   ('Matriz_mesangial_mesangiolise', 0, 100.0),
   ('Celularidade_mesangial_diminuida', 0, 100.0),
   ('Membrana_basal_glomerular_duplicacao_e_espiculas', 0, 100.0),
   ('Membrana_basal_glomerular_duplicacao_espiculas_e_ruptura', 0, 100.0),
   ('Membrana_basal_glomerular_espessamento', 0, 100.0),
   ('Membrana_basal_glomerular_espiculas', 0, 100.0),
   ('Membrana_basal_glomerular_ruptura_buracos_e_duplicacao', 0, 100.0),
   ('Membrana_basal_glomerular_ruptura_e_duplicacao', 0, 100.0),
   ('Membrana_basal_glomerular_ruptura_espiculas_e_duplicacao', 0, 100.0),
   ('Espaco_de_Bowman_adesao_e_crescentes', 0, 100.0),
   ('Espaco_de_Bowman_capsular_drops', 0, 100.0),
   ('Espaco_de_Bowman_crescentes', 0, 100.0),
   ('Espaco_de_Bowman_crescentes_em_mais_de_50%', 0, 100.0),
   ('Tufo_glomerular_congesto', 0, 100.0),
   ('Tufo_glomerular_esclerose_segmentar_e_global

# Printando os Rótulos de cada Grupo

In [18]:
def print_rotulos(rotulos):
    
    for rotulo in rotulos:
        print('Grupo:', rotulo[0][0])
        
        print('Atributos Rótulo: ')
        for i in range(len(rotulo[1])):
            if((rotulo[1][i][0] == 'Media') | (rotulo[1][i][0] == 'Desvio') | (rotulo[1][i][0] == 'Descritor de Suavidade Relativa') | (rotulo[1][i][0] == 'Terceiro Momento') | (rotulo[1][i][0] == 'Uniformidade') | (rotulo[1][i][0] == 'Entropia')):
                print('-> ', rotulo[1][i][0],'_',rotulo[1][i][1])
                print('    Porcentagem de influência do atributo: ', rotulo[1][i][2], '%')
                
            elif(rotulo[1][i][1] == 1):
                print('-> ', rotulo[1][i][0])
                print('    Porcentagem de influência do atributo: ', rotulo[1][i][2], '%')
                
        print('\n')

In [19]:
print_rotulos(rotulos)

Grupo: 0
Atributos Rótulo: 
->  Necrose_tubular_aguda_ausente
    Porcentagem de influência do atributo:  100.0 %
->  Cristais_ausente
    Porcentagem de influência do atributo:  100.0 %
->  Calcificacao_ausente
    Porcentagem de influência do atributo:  100.0 %
->  Tubulite_ausente
    Porcentagem de influência do atributo:  100.0 %
->  Atrofia_tubular_ausente
    Porcentagem de influência do atributo:  100.0 %
->  Graduacao_de_atrofia_ausente
    Porcentagem de influência do atributo:  100.0 %
->  Vasculite_ausente
    Porcentagem de influência do atributo:  100.0 %
->  Microangiopatia_trombotica_ausente
    Porcentagem de influência do atributo:  100.0 %
->  Trombos_ausente
    Porcentagem de influência do atributo:  100.0 %
->  Necrose_Fibrinoide_ausente
    Porcentagem de influência do atributo:  100.0 %
->  Vermelho_congo_nao_realizado
    Porcentagem de influência do atributo:  100.0 %


Grupo: 1
Atributos Rótulo: 
->  Necrose_tubular_aguda_ausente
    Porcentagem de influência